In [ ]:
!pip install streamlit langchain_core langchain_community ollama
from IPython.display import clear_output
clear_output()

In [ ]:
import streamlit as st
from langchain_community.chat_models import ChatOllama
from langchain_community.utilities import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate
import ollama as Ollama
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool
import sqlite3

In [ ]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
clear_output()

#Create a Python script to start the Ollama API server in a seperate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
  os.environ['OLLAMA_HOST'] = '127.0.0.1:11434'
  os.environ['OLLAMA_ORIGINS'] = '*'
  subprocess.Popen(['ollama', 'serve'])

In [ ]:
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [ ]:
!ollama pull llama3.1:70b
clear_output()

In [ ]:
!ollama pull llama3.1:8b
clear_output()

In [ ]:
def connectDatabase(url):
    response = requests.get(url)
    sql_script = response.text

    connection = sqlite3.connect(":memory:", check_same_thread=False)
    connection.executescript(sql_script)
    engine = create_engine(
        "sqlite://",
        creator=lambda: connection,
        poolclass=StaticPool,
        connect_args={"check_same_thread": False})
    st.session_state.db = SQLDatabase(engine)

def runQuery(query):
  if st.session_state.db:
    return st.session_state.db.run(query)
  else:
    "Please connect to database"


def getDatabaseSchema():
  if st.session_state.db:
    return st.session_state.db.get_table_info()
  else:
    "Please connect to database"


def getQueryFromLLM(llm, question, max_iteration=10):
    template = """below is the schema of SQLite database, read the schema carefully about the table and column names. Also take care of table or column name case sensitivity.
    Finally answer user's question in the form of SQL query.

    {schema}

    please only provide the SQL query and nothing else

    for example:
    question: how many albums we have in database
    SQL query: SELECT COUNT(*) FROM album
    question: how many customers are from Brazil in the database ?
    SQL query: SELECT COUNT(*) FROM customer WHERE country=Brazil

    your turn :
    question: {question}
    SQL query :
    please only provide the SQL query and nothing else
    """
    prompt = ChatPromptTemplate.from_template(template)  # Define prompt outside the loop
    chain = prompt | llm  # Define initial chain outside the loop
    i = max_iteration
    response = None  # Initialize response to None

    while i>0:
        try:
            response = chain.invoke({
                "question": question,
                "schema": getDatabaseSchema(),
                "error" : ""
            })
            # Attempt to execute the query to check its validity
            result = runQuery(response.content)
            # If execution is successful, break the loop
            break
        except Exception as error:
            # If an error occurs, feed the error message back to the LLM
            print(f"Error encountered: {error}")
            template = """Previous query attempt failed with error: {error}.
                           Please try generating a different SQL query for the question: {question}.
                           Here is the database schema for reference: {schema}
                           SQL query: """
            prompt = ChatPromptTemplate.from_template(template)
            chain = prompt | llm
            response = chain.invoke({  # Invoke the chain with the error message
                "question": question,
                "schema": getDatabaseSchema(),
                "error": str(error)  # Pass the error message to the prompt
            })
            i -= 1

    if response:  # Check if response has been assigned a value
      # Check if the response indicates failure
      if "Failed to generate a valid query." in response.content:
        return None  # Return None to signal query generation failure
      else:
        return response.content
    else:
        return None  # Return None if no response was generated


def getResponseForQueryResult(llm, question, query, result):
    template2 = """below is the schema of SQLite database, read the schema carefully about the table and column names of each table.
    Also look into the conversation if available
    Finally write a response in natural language by looking into the conversation and result.

    {schema}

    Here are some example for you:
    question: how many albums we have in database
    SQL query: SELECT COUNT(*) FROM album;
    Result : [(34,)]
    Response: There are 34 albums in the database.

    question: how many users we have in database
    SQL query: SELECT COUNT(*) FROM customer;
    Result : [(59,)]
    Response: There are 59 users in the database.

    question: how many users above are from india we have in database
    SQL query: SELECT COUNT(*) FROM customer WHERE country=india;
    Result : [(4,)]
    Response: There are 4 users in the database.

    your turn to write response in natural language from the given result :
    question: {question}
    SQL query : {query}
    Result : {result}
    Response:
    """

    prompt2 = ChatPromptTemplate.from_template(template2)
    chain2 = prompt2 | llm

    response = chain2.invoke({
        "question": question,
        "schema": getDatabaseSchema(),
        "query": query,
        "result": result
    })

    return response.content

In [ ]:
st.set_page_config(
    page_icon="🤖",
    page_title="Chat with SQL DB",
    layout="centered"
)
st.title("Chat with SQL DB  🤖")

question = st.chat_input('Chat with an SQL database')

if "chat" not in st.session_state or st.sidebar.button("Clear message history"):
    st.session_state["chat"] = [{"role": "assistant", "content": "This is a SQL Database Chain that can answer questions about a database. \
        Please input your question below:"}]

# Display chat messages from history on app rerun
for chat in st.session_state.chat:
    st.chat_message(chat['role']).markdown(chat['content'])

models = [model["name"] for model in Ollama.list()["models"]]
with st.sidebar:
    st.title('Connect to a Database & Select an LLM')
    st.text_input(label="Database", key="database", value="ex: https://raw.githubusercontent.com/....sql")
    st.sidebar.selectbox(label ="Model", key="model", options=models)
    connectBtn = st.button("Connect")

if connectBtn:
    connectDatabase(url=st.session_state.database)
    llm = ChatOllama(model=st.session_state.model, temperature=0.1)
    st.success("Database connected")

if question:
    if "db" not in st.session_state:
        st.error('Please connect to a database first.')
    else:
        st.session_state.chat.append({
            "role": "user",
            "content": question
        })

        query = getQueryFromLLM(llm, question)
        print(query)
        result = runQuery(query)
        print(result)
        response = getResponseForQueryResult(llm, question, query, result)
        st.session_state.chat.append({
            "role": "assistant",
            "content": response
        })